In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import csv
import os

In [2]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print('Device: ' + device)

Device: cpu


In [40]:
batch_size = 8
n = 12

x_file_path = 'Datasets/kryptonite-' + str(n) + '-X.npy'
y_file_path = 'Datasets/kryptonite-' + str(n) + '-y.npy'

In [41]:
x_raw = torch.tensor(np.load(x_file_path), dtype=torch.float32)
y_raw = torch.tensor(np.load(y_file_path), dtype=torch.float32)

# print(torch.unique(y_raw))

row_count = x_raw.shape[0]
print(f'Row Count: {row_count}')

x_train, x_val = torch.tensor_split(x_raw, [round(row_count * 0.8)], dim=0)
y_train, y_val = torch.tensor_split(y_raw, [round(row_count * 0.8)], dim=0)

train_dataset = torch.utils.data.TensorDataset(x_train.to(device), y_train.to(device))
val_dataset = torch.utils.data.TensorDataset(x_val.to(device), y_val.to(device))

loaders = {
            'train' : torch.utils.data.DataLoader(train_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True, 
                                                num_workers=1),
            
            'validation'  : torch.utils.data.DataLoader(val_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False, 
                                                num_workers=1),
        }


Row Count: 24000


In [42]:
model_struct = [n, 15, 10, 4, 1]
dropout = [0.0, 0.0, 0.0]

class kryptonite_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(model_struct[0], model_struct[1])
        self.act1 = nn.ReLU()
        # self.dropout1 = nn.Dropout(dropout[0])
        self.layer2 = nn.Linear(model_struct[1], model_struct[2])
        self.act2 = nn.ReLU()
        # self.dropout2 = nn.Dropout(dropout[1])
        self.layer3 = nn.Linear(model_struct[2], model_struct[3])
        self.act3= nn.ReLU()
        # self.dropout3 = nn.Dropout(dropout[2])
        self.output = nn.Linear(model_struct[3], model_struct[4])
        self.act_output = nn.Sigmoid()

        # nn.init.xavier_normal_(self.layer1.weight)
        # nn.init.zeros_(self.layer1.bias)

        # nn.init.xavier_normal_(self.layer2.weight)
        # nn.init.zeros_(self.layer2.bias)

        # nn.init.xavier_normal_(self.layer3.weight)
        # nn.init.zeros_(self.layer3.bias)

        # nn.init.xavier_normal_(self.output.weight)
        # nn.init.zeros_(self.output.bias)
 
    def forward(self, x):
        x = self.act1(self.layer1(x))
        # x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        # x = self.dropout2(x)
        x = self.act3(self.layer3(x))
        # x = self.dropout3(x)
        # x = self.act4(self.layer4(x))
        # x = self.dropout4(x)
        x = self.act_output(self.output(x))
        return x
 
model = kryptonite_nn()
# print(model)

In [ ]:
loss_func = 'BCE'           # Just used for recording in csv file
optim_func = 'Adadelta'     # Just used for recording in csv file
lr = 0.3

loss_fn = nn.BCELoss()
optimizer = optim.Adadelta(model.parameters(), lr=lr)

n_epochs = 25
 
for epoch in range(n_epochs):
    loss_arr = []
    for data in loaders['train']:
        x_vals, labels = data
        optimizer.zero_grad()
        label_pred = model(x_vals)
        labels = torch.reshape(labels, (batch_size, 1))
        loss = loss_fn(label_pred, labels)
        loss.backward()
        optimizer.step()
        loss_arr.append(loss.item())

    print(f'Finished epoch {epoch}, epoch loss {np.average(loss_arr)}')

Finished epoch 0, epoch loss 0.6931842321654161
Finished epoch 1, epoch loss 0.6931275175511837
Finished epoch 2, epoch loss 0.693038446828723
Finished epoch 3, epoch loss 0.6927361974865198
Finished epoch 4, epoch loss 0.6905325878659884
Finished epoch 5, epoch loss 0.6790708159903686
Finished epoch 6, epoch loss 0.6563159806778034
Finished epoch 7, epoch loss 0.597315891298155
Finished epoch 8, epoch loss 0.5142883480216066
Finished epoch 9, epoch loss 0.45554223915872477
Finished epoch 10, epoch loss 0.41725617138668897
Finished epoch 11, epoch loss 0.39114017246291044
Finished epoch 12, epoch loss 0.3753508759910862
Finished epoch 13, epoch loss 0.36789071863982825
Finished epoch 14, epoch loss 0.3628425187058747
Finished epoch 15, epoch loss 0.3575941683965114
Finished epoch 16, epoch loss 0.35293609926632297
Finished epoch 17, epoch loss 0.34754516202723607
Finished epoch 18, epoch loss 0.3440100177540444
Finished epoch 19, epoch loss 0.3401821779467476
Finished epoch 20, epoch l

In [36]:
train_accuracy = np.array([])
with torch.no_grad():
    for data in loaders['train']:
        x_vals, labels = data
        output = model.forward(x_vals)

        train_accuracy = np.concatenate((train_accuracy, torch.eq(torch.flatten(output.round()), labels).numpy() ))

train_accuracy = train_accuracy.mean()

print(f"Training Accuracy: {train_accuracy}")

Training Accuracy: 0.9511111111111111


In [37]:
y_pred = np.array([])
with torch.no_grad():
    for data in loaders['validation']:
        x_vals, labels = data
        output = model.forward(x_vals)
        # y_pred = np.concatenate((y_pred, output.numpy().flatten()))
        y_pred = np.concatenate((y_pred, torch.eq(torch.flatten(output.round()), labels).numpy() ))

val_accuracy = y_pred.mean()        
# val_accuracy = (y_pred.round() == y_val).float().mean()
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.9508333333333333


In [38]:
# Save model params to csv file

csv_file_path = 'nn_model_record.csv'

fields = ['n', 'train_acc', 'val_acc', 'optim_func', 'learning_rate', 'loss_func', 'epochs', 'batch_size', 'model_struct', 'dropout']

if not os.path.isfile(csv_file_path):
    with open(csv_file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)

with open(csv_file_path, mode='a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([n, round(train_accuracy.item(), 3), round(val_accuracy.item(), 3), optim_func, lr, loss_func, n_epochs, batch_size, model_struct, dropout])

    

In [39]:
torch.save(model.state_dict(), 'Saved_Models/n-' + str(n) + '-' + str(round(val_accuracy.item(), 3))[2:] + '.pth')